In [1]:
import numpy
import toyplot

canvas = toyplot.Canvas(width=600, height=600)
canvas.cartesian(grid=(2,2,0), label="Cartesian Coordinates").plot(numpy.linspace(0, 1)**2)
canvas.numberline(grid=(2,2,1), label="Numberline Coordinates").scatterplot(numpy.random.normal(size=100))
canvas.table(grid=(2,2,2), label="Table Coordinates", data = numpy.random.random((4, 3)));

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t8a7a59071442422985b631cef6817c46" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 600.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0 250.0 L 54.0 249.9167013744273 L 58.0 249.66680549770928 L 62.0 249.2503123698459 L 66.0 248.66722199083716 L 70.0 247.91753436068305 L 74.0 247.0012494793836 L 78.0 245.91836734693877 L 82.0 244.6688879633486 L 86.0 243.25281132861306 L 90.0 241.6701374427322 L 94.0 239.92086630570594 L 98.0 238.00499791753435 L 102.0 235.92253227821743 L 106.0 233.67346938775512 L 110.0 231.25780924614745 L 114.0 228.67555185339444 L 118.0 225.92669720949604 L 122.0 223.0112453144523 L 126.0 219.92919616826322 L 130.0 216.6805497709288 L 134.0 213.26530612244898 L 138.0 209.68346522282383 L 142.0 205.93502707205332 L 146.0 202.01999167013744 L 150.0 197.9383590170762 L 154.0 193.69012911286964 L 158.0 189.2753019575177 L 162.0 184.69387755102045 L 166.0 179.94585589337777 L 170.0 175.03123698458978 L 174.0 169.9500208246564 L 178.0 164.7022074135777 L 182.0 159.2877967513536 L 186.0 153.70678883798416 L 190.0 147.9591836734694 L 194.0 142.04498125780927 L 198.0 135.96418159100375 L 202.0 129.7167846730529 L 206.0 123.30279050395671 L 210.0 116.72219908371514 L 214.0 109.97501041232822 L 218.0 103.06122448979593 L 222.0 95.9808413161183 L 226.0 88.73386089129534 L 230.0 81.32028321532698 L 234.0 73.74010828821326 L 238.0 65.99333610995419 L 242.0 58.079966680549774 L 246.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 0.0 0.5 1.0 Cartesian Coordinates -2 0 2 4 Numberline Coordinates Table Coordinates 0 1 2 0 . 420852 0 . 797511 0 . 379102 0 . 715057 0 . 436308 0 . 927374 0 . 656393 0 . 78502 0 . 98881 0 . 511233 0 . 894884 0 . 150652

Naturally, some coordinate systems - such as Cartesian - allow you to specify additional, axis-specific labels:

In [2]:
canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian(label="Cartesian Coordinates", xlabel="Days", ylabel="Users")
axes.plot(numpy.linspace(0, 1)**2);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t977e944485e64d1e82f3f25c6ce3d0ef" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 300.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0 250.0 L 54.0 249.9167013744273 L 58.0 249.66680549770928 L 62.0 249.2503123698459 L 66.0 248.66722199083716 L 70.0 247.91753436068305 L 74.0 247.0012494793836 L 78.0 245.91836734693877 L 82.0 244.6688879633486 L 86.0 243.25281132861306 L 90.0 241.6701374427322 L 94.0 239.92086630570594 L 98.0 238.00499791753435 L 102.0 235.92253227821743 L 106.0 233.67346938775512 L 110.0 231.25780924614745 L 114.0 228.67555185339444 L 118.0 225.92669720949604 L 122.0 223.0112453144523 L 126.0 219.92919616826322 L 130.0 216.6805497709288 L 134.0 213.26530612244898 L 138.0 209.68346522282383 L 142.0 205.93502707205332 L 146.0 202.01999167013744 L 150.0 197.9383590170762 L 154.0 193.69012911286964 L 158.0 189.2753019575177 L 162.0 184.69387755102045 L 166.0 179.94585589337777 L 170.0 175.03123698458978 L 174.0 169.9500208246564 L 178.0 164.7022074135777 L 182.0 159.2877967513536 L 186.0 153.70678883798416 L 190.0 147.9591836734694 L 194.0 142.04498125780927 L 198.0 135.96418159100375 L 202.0 129.7167846730529 L 206.0 123.30279050395671 L 210.0 116.72219908371514 L 214.0 109.97501041232822 L 218.0 103.06122448979593 L 222.0 95.9808413161183 L 226.0 88.73386089129534 L 230.0 81.32028321532698 L 234.0 73.74010828821326 L 238.0 65.99333610995419 L 242.0 58.079966680549774 L 246.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 Days 0.0 0.5 1.0 Users Cartesian Coordinates

## Coordinate System Text

Another option for labelling a figure is to insert text using the same domain as the data.  For example, we can label individual series in a plot:

In [3]:
def series(x):
    return numpy.cumsum(numpy.random.normal(loc=0.05, size=len(x)))

numpy.random.seed(1234)
x = numpy.arange(100)
y = numpy.column_stack([series(x) for i in range(5)])

In [4]:
label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px"}
canvas, axes, mark = toyplot.plot(x, y)
for i in range(y.shape[1]):
    axes.text(x[-1], y[-1,i], "Series %s" % i, style=label_style)

<svg class="toyplot-canvas-Canvas" height="600px" id="tbbb24e62edf84d51a74aa528ae27f318" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0 343.3300205953749 L 54.59959732549238 358.7920369979883 L 59.19919465098476 338.69902399829283 L 63.79879197647714 342.2583705684162 L 68.39838930196953 351.3459033202555 L 72.99798662746191 338.645870513082 L 77.5975839529543 326.3195159397324 L 82.19718127844668 334.2678325321839 L 86.79677860393906 333.37754329294927 L 91.39637592943143 363.0918754077105 L 95.99597325492383 346.8295026460248 L 100.5955705804162 332.70949416307025 L 105.19516790590859 319.1128731325027 L 109.79476523140096 345.82647843101597 L 114.39436255689336 349.6761738365487 L 118.99395988238572 348.96988796792573 L 123.59355720787812 342.7977774199124 L 128.1931545333705 338.2025479353999 L 132.79275185886289 319.6211964170079 L 137.39234918435525 339.90662258741213 L 141.99194650984765 341.975220530528 L 146.59154383534002 350.18705885629316 L 151.1911411608324 346.88831609324103 L 155.79073848632478 338.71076911964946 L 160.39033581181715 320.1701622775058 L 164.98993313730955 325.8524088143114 L 169.58953046280195 316.0200090382306 L 174.18912778829431 339.9660090171619 L 178.7887251137867 341.76983923118337 L 183.38832243927908 326.74156137410824 L 187.98791976477148 331.4553439190562 L 192.58751709026387 326.2049539388292 L 197.18711441575624 311.33103683791296 L 201.7867117412486 296.4793486175726 L 206.38630906674095 284.0970412494765 L 210.98590639223335 285.07399555091905 L 215.58550371772574 282.70636005311985 L 220.1851010432181 286.4031590107421 L 224.7846983687105 274.3195632758751 L 229.38429569420288 241.2407065541016 L 233.98389301969527 239.53050351805496 L 238.58349034518764 246.52915869095634 L 243.18308767068004 245.36179987328694 L 247.78268499617243 272.8034333998005 L 252.3822823216648 268.76788040910606 L 256.9818796471572 280.2481881469161 L 261.58147697264957 281.4243947407773 L 266.18107429814194 280.4989687201691 L 270.7806716236343 269.5843416474706 L 275.38026894912673 265.98953487895074 L 279.9798662746191 253.91496338192485 L 284.57946360011147 272.83038673960164 L 289.1790609256039 291.15175252455737 L 293.77865825109626 291.8417742721321 L 298.37825557658863 298.59374362393095 L 302.97785290208105 299.875986880472 L 307.5774502275734 294.40087550655176 L 312.1770475530658 294.2045541985582 L 316.7766448785582 285.86033106372054 L 321.3762422040506 264.23664291281534 L 325.97583952954295 276.7614998566594 L 330.5754368550353 277.0372049614677 L 335.17503418052775 272.18616357938197 L 339.7746315060201 274.33407127479313 L 344.3742288315125 259.64686559487825 L 348.97382615700485 291.4992116248591 L 353.5734234824972 263.30375797759905 L 358.1730208079896 278.11063159573075 L 362.7726181334819 274.56169953634054 L 367.3722154589744 264.3340472500366 L 371.9718127844667 274.3003518436393 L 376.57141010995906 267.3611364932593 L 381.1710074354515 257.1401569864081 L 385.77060476094385 249.3682213985208 L 390.3702020864362 261.2428464510297 L 394.9697994119286 233.35583570526822 L 399.569396737421 229.60255736184212 L 404.1689940629134 244.54532372648018 L 408.76859138840575 235.30342889360318 L 413.3681887138982 234.09039110820942 L 417.96778603939055 227.11956552076202 L 422.5673833648829 274.7332460658253 L 427.1669806903753 256.1526070439136 L 431.7665780158677 253.40664414882914 L 436.3661753413601 250.49943120200186 L 440.96577266685244 255.65032603189297 L 445.56536999234487 244.57369323610104 L 450.16496731783724 230.54890055625543 L 454.7645646433296 226.20106985300572 L

Note that we are using the last point in each series as the anchor for the corresponding label - by default, Toyplot renders text centered on its anchor, so in this case we've chosen a text style that left-aligns the text and offsets it slightly to avoid overlapping the data.

## Canvas Text

When adding text to axes, you specify the text coordinates using the same domain as your data.  Naturally, this limits the added text to the bounds defined by the axes.  For the ultimate in labeling flexibility, you can add text to the canvas directly, using canvas units, outside and/or overlapping coordinate systems:

In [5]:
label_style={"font-size":"18px", "font-weight":"bold"}

canvas = toyplot.Canvas(width=600, height=300)
canvas.cartesian(grid=(1,2,0)).plot(numpy.linspace(1, 0)**2)
canvas.cartesian(grid=(1,2,1), yshow=False).plot(numpy.linspace(0, 1)**2)
canvas.text(300, 120, "This label overlaps two sets of axes!", style=label_style);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="td4b0757117f14a6da2d9342edd1ae876" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 300.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0 50.0 L 54.0 58.079966680549774 L 58.0 65.99333610995419 L 62.0 73.74010828821322 L 66.0 81.32028321532694 L 70.0 88.7338608912953 L 74.0 95.9808413161183 L 78.0 103.0612244897959 L 82.0 109.97501041232817 L 86.0 116.72219908371511 L 90.0 123.30279050395669 L 94.0 129.7167846730529 L 98.0 135.96418159100375 L 102.0 142.04498125780924 L 106.0 147.9591836734694 L 110.0 153.70678883798413 L 114.0 159.2877967513536 L 118.0 164.70220741357767 L 122.0 169.9500208246564 L 126.0 175.03123698458975 L 130.0 179.94585589337777 L 134.0 184.69387755102045 L 138.0 189.27530195751768 L 142.0 193.69012911286964 L 146.0 197.9383590170762 L 150.0 202.01999167013744 L 154.0 205.9350270720533 L 158.0 209.68346522282383 L 162.0 213.265306122449 L 166.0 216.68054977092876 L 170.0 219.92919616826322 L 174.0 223.0112453144523 L 178.0 225.926697209496 L 182.0 228.6755518533944 L 186.0 231.25780924614742 L 190.0 233.6734693877551 L 194.0 235.9225322782174 L 198.0 238.00499791753435 L 202.0 239.92086630570594 L 206.0 241.67013744273217 L 210.0 243.25281132861306 L 214.0 244.6688879633486 L 218.0 245.91836734693877 L 222.0 247.00124947938357 L 226.0 247.91753436068305 L 230.0 248.66722199083716 L 234.0 249.2503123698459 L 238.0 249.66680549770925 L 242.0 249.9167013744273 L 246.0 250.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 0.0 0.5 1.0 <path d="M 350.0 250.0 L 354.0 249.9167013744273 L 358.0 249.66680549770928 L 362.0 249.2503123698459 L 366.0 248.66722199083716 L 370.0 247.91753436068305 L 374.0 247.0012494793836 L 378.0 245.91836734693877 L 382.0 244.6688879633486 L 386.0 243.25281132861306 L 390.0 241.6701374427322 L 394.0 239.92086630570594 L 398.0 238.00499791753435 L 402.0 235.92253227821743 L 406.0 233.67346938775512 L 410.0 231.25780924614745 L 414.0 228.67555185339444 L 418.0 225.92669720949604 L 422.0 223.0112453144523 L 426.0 219.92919616826322 L 430.0 216.6805497709288 L 434.0 213.26530612244898 L 438.0 209.68346522282383 L 442.0 205.93502707205332 L 446.0 202.01999167013744 L 450.0 197.9383590170762 L 454.0 193.69012911286964 L 458.0 189.2753019575177 L 462.0 184.69387755102045 L 466.0 179.94585589337777 L 470.0 175.03123698458978 L 474.0 169.9500208246564 L 478.0 164.7022074135777 L 482.0 159.2877967513536 L 486.0 153.70678883798416 L 490.0 147.9591836734694 L 494.0 142.04498125780927 L 498.0 135.96418159100375 L 502.0 129.7167846730529 L 506.0 123.30279050395671 L 510.0 116.72219908371514 L 514.0 109.97501041232822 L 518.0 103.06122448979593 L 522.0 95.9808413161183 L 526.0 88.73386089129534 L 530.0 81.32028321532698 L 534.0 73.74010828821326 L 538.0 65.99333610995419 L 542.0 58.079966680549774 L 546.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 This label overlaps two sets of axes!

... remember when placing labels directly on the canvas that, unlike Cartesian coordinates, canvas coordinates increase from top-to-bottom.

In [6]:
data = toyplot.data.cars()

colormap = toyplot.color.brewer.map(
    name="BlueGreenBrown",
    reverse=True,
    domain_min = data["MPG"].min(),
    domain_max = data["MPG"].max(),
)
canvas = toyplot.Canvas(width=600, height=400)
axes = canvas.cartesian(xlabel="Year", ylabel="Horsepower", margin=75)
axes.scatterplot(
    data["Year"],
    data["Horsepower"],
    color=(data["MPG"], colormap),
    size=8,
    mstyle={"stroke":"black", "stroke-opacity":0.3}
)
axes.color_scale(colormap, label="MPG");

<svg class="toyplot-canvas-Canvas" height="400.0px" id="tadf26716e7bf47539dcf1d5702bb3ebf" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 400.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 70 74 78 82 Year 50 100 150 200 250 Horsepower 10 20 30 40 50 MPG

Note that a colormap must be explicitly specified when creating a color scale - this is necessary to avoid ambiguity when a single coordinate system contains multiple visualizations or data series.

In [7]:
canvas = toyplot.Canvas(width=600, height=400)

axes = canvas.cartesian(bounds=("10%", "45%", "10%", "65%"), xlabel="Year", ylabel="Horsepower")
axes.scatterplot(
    data["Year"],
    data["Horsepower"],
    color=(data["MPG"], colormap),
    size=8,
    mstyle={"stroke":"black", "stroke-opacity":0.3}
)

axes = canvas.cartesian(bounds=("-45%", "-10%", "10%", "65%"), xlabel="Weight", ylabel="Horsepower")
axes.y.spine.position="high"
axes.scatterplot(
    data["Weight"],
    data["Horsepower"],
    color=(data["MPG"], colormap),
    size=8,
    mstyle={"stroke":"black", "stroke-opacity":0.3}
)

canvas.color_scale(colormap, bounds=("10%", "-10%", "75%", "90%"), label="MPG");


<svg class="toyplot-canvas-Canvas" height="400.0px" id="tb1ccb2442afc4918b7d3052aee89cee7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 400.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 70 74 78 82 Year 50 100 150 200 250 Horsepower 2000 3000 4000 5000 Weight 50 100 150 200 250 Horsepower 10 20 30 40 50 MPG

Note that when manually adding a color scale to a canvas, you can orient it any way you like (including diagonally!) by explicitly specifying the endpoints in canvas coordinates:

In [8]:
colormap = toyplot.color.brewer.map(
    name="Spectral",
    domain_min=0,
    domain_max=1,
)
canvas = toyplot.Canvas(width=400)
canvas.color_scale(colormap, x1=50, y1=-50, x2=50, y2=50, label="Bottom to Top")
canvas.color_scale(colormap, x1=150, y1=50, x2=150, y2=-50, label="Top to Bottom")
canvas.color_scale(colormap, x1=200, y1=150, x2=350, y2=150, label="Left to Right")
canvas.color_scale(colormap, x1=350, y1=250, x2=200, y2=250, label="Right to Left");

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t00ef50ccace540bba53b867c48517991" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0 Bottom to Top 0.0 0.5 1.0 Top to Bottom 0.0 0.5 1.0 Left to Right 0.0 0.5 1.0 Right to Left

## Canvas Legends

Last-but-not-least, Toyplot provides basic support for graphical legends:

In [18]:
observations = numpy.random.power(2, size=(50, 50))

x = numpy.arange(len(observations))

boundaries = numpy.column_stack(
    (numpy.min(observations, axis=1),
     numpy.percentile(observations, 25, axis=1),
     numpy.percentile(observations, 50, axis=1),
     numpy.percentile(observations, 75, axis=1),
     numpy.max(observations, axis=1)))

color = ["blue", "blue", "red", "red"]
opacity = [0.1, 0.2, 0.2, 0.1]

canvas = toyplot.Canvas(600, 400)
axes = canvas.cartesian(bounds=(50, -50, 100, -50))
fill = axes.fill(x, boundaries, color=color, opacity=opacity)
mean = axes.plot(x, numpy.mean(observations, axis=1), color="blue")

canvas.legend([
    ("Mean", mean),
    ("Quartiles", fill),
    ],
    corner=("top", 30, 100, 50),
    );


<svg class="toyplot-canvas-Canvas" height="400.0px" id="t6a59276366434a8284d24b50e142000a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 400.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="50.0,296.22203320886547 60.0,334.57287422193207 70.0,307.7269493701809 80.0,326.99779620948607 90.0,328.69611034316614 100.0,340.9625074793732 110.0,325.35401942050106 120.00000000000001,324.20302163048615 130.0,303.82829825803736 140.0,321.63846394438195 150.0,298.8336653758565 160.0,305.6754804690866 170.0,320.22756649489844 180.0,314.05842836426893 190.00000000000003,331.20146140668913 200.0,299.28940496141786 210.0,323.2028432133392 220.0,331.147620023446 230.0,306.7777152421788 240.0,293.5352540393043 250.0,329.72868291630135 260.0,297.9946892873792 270.0,339.34088059000015 280.0,308.15511164417325 290.0,302.9760355363463 300.0,319.8573478839084 310.0,314.459750107612 320.0,336.9996544511921 330.00000000000006,327.66841354860884 340.0,304.0430060819167 350.0,318.5931344325216 360.0,324.52459385691526 370.0,335.730170025791 380.0,321.9853235717189 390.0,337.8860436844375 400.0,323.40470209212407 410.0,307.80574511700183 420.0,332.8225135285272 430.0,342.7653034047287 440.0,324.11688912309546 450.0,289.7097401712342 460.0,326.2191826294631 470.0,315.5637138363867 480.0,299.5449377776311 490.0,340.1526582791077 500.0,336.0835541776048 510.0,318.1587180145524 520.0,318.8044193069612 530.0,301.39434064069724 540.0,317.3959787397536 540.0,202.75226322100372 530.0,220.88677067915918 520.0,225.32319424349575 510.0,202.41021814295283 500.0,215.103470644597 490.0,231.22474314589005 480.0,212.22162553335215 470.0,236.17777876825735 460.0,230.72600201533328 450.0,218.54062724935613 440.0,216.50929869695258 430.0,261.7708348782758 420.0,228.8703986915272 410.0,197.72814474571047 400.0,213.89812538983318 390.0,237.95206439625528 380.0,225.02288424787886 370.0,235.0947609284751 360.0,207.27556031735656 350.0,208.90441213335356 340.0,216.3031763587864 330.00000000000006,245.41691609624348 320.0,216.1071816627663 310.0,217.26289201173384 300.0,229.886939739903 290.0,231.7756167015857 280.0,220.33091706732276 270.0,247.23377691545946 260.0,212.0617290762765 250.0,234.26144053901788 240.0,209.0247658957421 230.0,219.67734526359612 220.0,247.23096261472276 210.0,212.136659935077 200.0,204.53465365300866 190.00000000000003,251.29674110134107 180.0,223.09544435383629 170.0,229.24727620854563 160.0,231.18909944172393 150.0,240.77521658747818 140.0,228.34500382731878 130.0,235.31802456108713 120.00000000000001,251.73569289973688 110.0,250.0839601247852 100.0,207.6064475050114 90.0,217.33033006719864 80.0,225.57996455404947 70.0,229.71279330975605 60.0,235.36113737690297 50.0,207.99670985555278" style="fill:rgb(0%,0%,100%);fill-opacity:1.0;opacity:0.1;stroke:none"> <polygon points="50.0,207.99670985555278 60.0,235.36113737690297 70.0,229.71279330975605 80.0,225.57996455404947 90.0,217.33033006719864 100.0,207.6064475050114 110.0,250.0839601247852 120.00000000000001,251.73569289973688 130.0,235.31802456108713 140.0,228.34500382731878 150.0,240.77521658747818 160.0,231.18909944172393 170.0,229.24727620854563 180.0,223.09544435383629 190.00000000000003,251.29674110134107 200.0,204.53465365300866 210.0,212.136659935077 220.0,247.23096261472276 230.0,219.67734526359612 240.0,209.0247658957421 250.0,234.26144053901788 260.0,212.0617290762765 270.0,247.23377691545946 280.0,220.33091706732276 290.0,231.7756167015857 300.0,229.886939739903 310.0,217.26289201173384 320.0,216.1071816627663 330.00000000000006,245.41691609624348 340.0,216.3031763587864 350.0,208.90441213335356 360.0,207.27556031735656 370.

In [10]:
x = numpy.linspace(0, 1)
y1 = (1 - x) ** 2
y2 = numpy.column_stack((1 - (x ** 2), x ** 2))

canvas = toyplot.Canvas(width=600, height=300)
m1 = canvas.cartesian(grid=(1,2,0), margin=25).scatterplot(x, y1, marker="o", color="rgb(255,0,0)")
m2 = canvas.cartesian(grid=(1,2,1), margin=25, yshow=False).scatterplot(x, y2, marker="s", color=["green", "blue"])

if True:
    canvas.legend([
        ("Experiment 1", m1),
        ("Experiment 2", m2.markers[0]),
        ("Experiment 3", m2.markers[1]),
        ],
        corner=("top", 100, 100, 70),
        );

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t300275d8f56c411686fc2811d138da76" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 300.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0 0.0 0.5 1.0 0.0 0.5 1.0 Experiment 1 Experiment 2 Experiment 3